In [1]:
test_data_path='data/test'

读入文件夹列表，每个文件夹包含一个帧序列，使用模型进行目标检测，将结果保存在同名txt文件中，文本内容形如：
{“res”:[[x,y,width,height],[x,y,width,height],[],...[x,y,width,height]}

In [2]:
import os

from ultralytics import YOLO

model=YOLO('runs/detect/train/weights/best.pt')

In [3]:
import json
import PIL


def location_transfer(img,pre):
    # 将yolo给出的预测结果转换为原图的坐标
    # 坐标格式为：左上角x，左上角y，宽，高，使用绝对像素值
    img=PIL.Image.open(img)
    size=img.size
    pre_new=[]
    x_left=int(pre[0]-pre[2]/2)
    y_left=int(pre[1]-pre[3]/2)
    img.close()
    return [x_left,y_left,int(pre[2]),int(pre[3])]
    
folder_list = os.listdir(test_data_path)
folder_list.sort()
def predictor(begin,end):
    model=YOLO('runs/detect/train/weights/best.pt')
    for folder in folder_list[begin:end]:
        file_list = os.listdir(test_data_path+'/'+folder)
        # 剔除json文件，按照文件名排序
        if 'IR_label.json' in file_list:
            file_list.remove('IR_label.json')
        file_list.sort()
        file_list=[test_data_path+'/'+folder+'/'+file for file in file_list]
        res=[]
        # 载入第一帧的标注
        res.append(json.load(open(os.path.join(test_data_path,folder_list[0],'IR_label.json')))['res'][0])
        for img in file_list[1:]:
            # 预测
            pre=model.predict(img)[0]
            # 转换坐标
            pre=pre.boxes.xywh.tolist()
            if len(pre)==0:
                res.append([])
            else:
                pre=pre[0]
                pre=location_transfer(img,pre)
                res.append(pre)
            del pre
        res={'res':res}
        with open(os.path.join('res',folder+'.txt'),'w+') as f:
            f.write(json.dumps(res))


image 1/1 /home/wzy/Git/Thermal-Dectect/data/test/20190926_103046_1_7/000002.jpg: 512x640 1 UAV, 71.7ms
Speed: 0.4ms preprocess, 71.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/wzy/Git/Thermal-Dectect/data/test/20190926_103046_1_7/000003.jpg: 512x640 1 UAV, 4.1ms
Speed: 0.7ms preprocess, 4.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/wzy/Git/Thermal-Dectect/data/test/20190926_103046_1_7/000004.jpg: 512x640 1 UAV, 4.3ms
Speed: 0.3ms preprocess, 4.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/wzy/Git/Thermal-Dectect/data/test/20190926_103046_1_7/000005.jpg: 512x640 1 UAV, 4.2ms
Speed: 0.4ms preprocess, 4.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/wzy/Git/Thermal-Dectect/data/test/20190926_103046_1_7/000006.jpg: 512x640 1 UAV, 3.9ms
Speed: 0.4ms preprocess, 3.9ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640

In [ ]:
import multiprocessing
p1=multiprocessing.Process(target=predictor,args=(0,10))
p2=multiprocessing.Process(target=predictor,args=(10,20))
p3=multiprocessing.Process(target=predictor,args=(20,30))
p4=multiprocessing.Process(target=predictor,args=(30,40))
p5=multiprocessing.Process(target=predictor,args=(40,50))
p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()